# Data preprocessing

## I- Modules imports

In [1]:
# Import the necessary modules
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import  when,col,lit,length,regexp_replace,trim, lower,udf,concat_ws,ArrayType,size,min, split,desc,expr,format_number
from pyspark.sql.types import  StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover,StringIndexer
from nltk import WordNetLemmatizer, download
import nlpaug.augmenter.word as naw
import numpy as np
#import nltk
from json import dump

## II- PySpark application and session creation

In [2]:
# Configure a Spark Capplication for data preprocessing
conf=SparkConf().setAppName('DataPreprocessing')\
.set("spark.driver.memory", '20g')
# Create the Spark Context
sc=SparkContext(conf=conf)
sc

24/06/26 04:54:25 WARN Utils: Your hostname, starias-T14s resolves to a loopback address: 127.0.1.1; using 10.152.186.157 instead (on interface wlp0s20f3)
24/06/26 04:54:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 04:54:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=DataPreprocessing>

In [3]:
# Create a Spark Session
spark=SparkSession(sc)
spark


## III- Preprocessing each dataset

We have five datasets to preprocess



### Dataset 1: Topic-labeled news dataset

[Link to the dataset (Kaggle)](https://www.kaggle.com/datasets/kotartemiy/topic-labeled-news-dataset)

#### A- Data exploration

This is a csv file with a header and with values delimited by semicolons(;).

In [4]:
# Reading the file to get a dataframe
df=spark.read.csv('input/input1.csv',header=True,inferSchema=True,sep=';')

# Print the number of records
print(df.count(),'rows at all')

# Print the schema of the data
df.printSchema()

# Show the data (first 20 rows)
df.show()

108774 rows at all
root
 |-- topic: string (nullable = true)
 |-- link: string (nullable = true)
 |-- domain: string (nullable = true)
 |-- published_date: timestamp (nullable = true)
 |-- title: string (nullable = true)
 |-- lang: string (nullable = true)

+-------+--------------------+--------------------+-------------------+--------------------+----+
|  topic|                link|              domain|     published_date|               title|lang|
+-------+--------------------+--------------------+-------------------+--------------------+----+
|SCIENCE|https://www.eurek...|      eurekalert.org|2020-08-06 13:59:45|A closer look at ...|  en|
|SCIENCE|https://www.pulse...|            pulse.ng|2020-08-12 15:14:19|An irresistible s...|  en|
|SCIENCE|https://www.expre...|       express.co.uk|2020-08-13 21:01:00|Artificial intell...|  en|
|SCIENCE|https://www.ndtv....|            ndtv.com|2020-08-03 22:18:26|Glaciers Could Ha...|  en|
|SCIENCE|https://www.thesu...|           thesun.ie|2020-

We remark that the dataset contains **6** fields and **108774** instances.

The fiedls are the following:

- ` topic`: What the new is about.
-` link`: The URL to the new
- `domain`: The domain name of the website where the new was collected
- `published_date`: The publication date of the new
- `title`: The title of the new
- `lang`: The language in which the new was written

As we are going to work with english news only, we must ensure that our news are in english. So let us list the languages of the news.

Let us group the data by language and count the number of occurences of each group.

In [5]:
df.select(df.lang).groupBy(df.lang).count().show()

+--------------------+------+
|                lang| count|
+--------------------+------+
|                  en|108771|
| Arrives August 2...|     1|
| A Groundbreaking...|     1|
| Saif Ali Khan's ...|     1|
+--------------------+------+



We see that there are `3` instances of the data which have wrong values of language.

The other have as value of language, ` en`, which means they are english news.

Let us look at the content of the three wrong instances.

In [6]:
df.select(df.title).filter(df.lang!='en').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                         |
+------------------------------------------------------------------------------------------------------------------------------+
|"Upcoming 'Control' DLC ""AWE"" Crosses Over With 'Alan Wake'                                                                 |
|"ASHBA Drops Highly Anticipated Debut Single ""Hypnotic"" (Feat. Cali Tucker) On August 14                                    |
|"""Arnab, The News Prostitute"": Ram Gopal Varma announces title of his next film to take 'clothes off' Arnab Goswami's facade|
+------------------------------------------------------------------------------------------------------------------------------+



The table shows that these instances are in english too.

Now let us move to the next section, which is features and colums renaming

#### B- Features selection and columns renaming

Among the 6 columns of our dataset, only 2 are relative to our task. There are `topic` and `title`.

Let us select only these columns and then let us rename them to `category` and `description`


In [7]:
# Feautures selection and column renaming

# Select the relevant columns
df=df.select([df.title,df.topic])\
.withColumnsRenamed({'topic':'category','title':'description'}) # Rename the columns

# Print the schema of the new dataframe
df.printSchema()

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)



Now the schema of our data is:

- `description`: The description of the data
- `category`: The category of the data

Let us move to the next section, whiich consists to handle missing values.

#### C- Handling missing values

First of all we need to identify the missing values

In [8]:
df.filter((df.category.isNull()) | (df.description.isNull())).count()

0

The previous code counts the number of null values in our dataframe. As the result is 0, there is no null value in our data.

#### D- Summary

Our preprocessed data now contains 108774 rows, and 2 colums which are `description` and `category`.

Let us copy it into anoter variable.

In [9]:
# Make a copy
df1=df
df1.show()
df1.printSchema()

+--------------------+--------+
|         description|category|
+--------------------+--------+
|A closer look at ...| SCIENCE|
|An irresistible s...| SCIENCE|
|Artificial intell...| SCIENCE|
|Glaciers Could Ha...| SCIENCE|
|Perseid meteor sh...| SCIENCE|
|NASA Releases In-...| SCIENCE|
|SpaceX, NASA Demo...| SCIENCE|
|Orbital space tou...| SCIENCE|
|Greenland's melti...| SCIENCE|
|NASA invites engi...| SCIENCE|
|Astronomers Detec...| SCIENCE|
|NASA Finds Ammoni...| SCIENCE|
|Russia targets re...| SCIENCE|
|Martian Night Sky...| SCIENCE|
|Energy-Efficient ...| SCIENCE|
|Scientists may ha...| SCIENCE|
|Tradeoff between ...| SCIENCE|
|2020's final Mars...| SCIENCE|
|In rare find, fos...| SCIENCE|
|Nasa SpaceX crew ...| SCIENCE|
+--------------------+--------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)



### Dataset 2: News categories dataset

[Link to the dataset (Kaggle)](https://www.kaggle.com/datasets/rmisra/news-category-dataset)

####  Data exploration

This is a json file.

In [10]:
# Read the file to get a dataframe
df=spark.read.json('input/input2.json')
# Print the number of records

print(df.count(),'rows at all')

# Print the schema of the data
df.printSchema()

# Show the data (first 20 rows)
df.show()

209527 rows at all
root
 |-- authors: string (nullable = true)
 |-- category: string (nullable = true)
 |-- date: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- link: string (nullable = true)
 |-- short_description: string (nullable = true)

+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|             authors|      category|      date|            headline|                link|   short_description|
+--------------------+--------------+----------+--------------------+--------------------+--------------------+
|Carla K. Johnson, AP|     U.S. NEWS|2022-09-23|Over 4 Million Am...|https://www.huffp...|Health experts sa...|
|      Mary Papenfuss|     U.S. NEWS|2022-09-23|American Airlines...|https://www.huffp...|He was subdued by...|
|       Elyse Wanshel|        COMEDY|2022-09-23|23 Of The Funnies...|https://www.huffp...|"Until you have a...|
|    Caroline Bologna|     PARENTING|2022-09-23|The Funniest Tw

We remark that the dataset contains **6** fields and **209527** instances.

The fiedls are the following:

- ` authors`: The authors of the new.
- `category`: The category of the new
- `date` the publication date of the new
- `headline`: The headline of the new
-` link`: The URL to the new
- `short_description`: A short description of the new

#### B- Features selection

Among the 6 columns of our dataset, only 3 are relative to our task. There are `category`, `headline` and `short_description`

Let us select only these columns

In [11]:
# Features selection

# Select the relevant columns
df=df.select([df.category,df.headline,df.short_description])

# Print the schema of the new dataframe
df.printSchema()

root
 |-- category: string (nullable = true)
 |-- headline: string (nullable = true)
 |-- short_description: string (nullable = true)



Now the schema of our data is:

- `category`: The category of the news
- `headline`: Rhe headline of the news
-`short_description`: A short description of the news

Let us move to the next section, whiich consists to handle missing values.

#### C- Handling missing values

First of all we need to identify the missing values

In [12]:
df.filter(df.category.isNull()).show()

+--------+--------+-----------------+
|category|headline|short_description|
+--------+--------+-----------------+
+--------+--------+-----------------+



Basing on the code we may say that ther is no missing value but that is wrong.

In fact, we have loaded our data from JSON but JSON file format is a semi-structured data format so without a predifined structure.

All the dat we have loaded from our file has been considerred s string, including dates and null values. So we need to look for null values with another method: He have to look for empty strings ie `''`

In [13]:
null_values_df=df.filter( (df.category=='') | (df.headline=='' ) | (df.short_description==''))
null_values_df.show()
null_values_df.count()

+--------------+--------------------+-----------------+
|      category|            headline|short_description|
+--------------+--------------------+-----------------+
|        COMEDY|   The Big Blue Wave|                 |
|    WORLD NEWS|Inside Rukban Cam...|                 |
|    WORLD NEWS|Syrian Refugees R...|                 |
|         TASTE|Your Guide To The...|                 |
|        COMEDY|    The Bechdel Test|                 |
|         WOMEN|Note: This conten...|                 |
|         WOMEN|Note: This conten...|                 |
|      POLITICS|Fired U.S. Attorn...|                 |
|HEALTHY LIVING|Love Needs To Be ...|                 |
|     WORLDPOST|A New American St...|                 |
|         GREEN|A Price On Carbon...|                 |
|      POLITICS|Geoff Cowan, Form...|                 |
|      POLITICS|Donald Trump Only...|                 |
|     WORLDPOST|Islamic Republic ...|                 |
|      BUSINESS|Price Wrong and L...|           

19713

The previous output shows that there `19713` instances containing missing values in our dataframe.

Firstly let us check if there are lines with null value everywhere.

In [14]:
# Find NULL values
null_values_df=df.filter( (df.category=='') & (df.headline=='' ) & (df.short_description==''))
null_values_df.show()
null_values_df.count()

+--------+--------+-----------------+
|category|headline|short_description|
+--------+--------+-----------------+
+--------+--------+-----------------+



24/06/26 04:54:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


0

So there is no null line in the data

Now let us find the NULL values in each colums

In [15]:
# Check NULL values in the column category
null_values_df=df.filter( (df.category=='') )
null_values_df.show()
null_values_df.count()

+--------+--------+-----------------+
|category|headline|short_description|
+--------+--------+-----------------+
+--------+--------+-----------------+



0

There is no NULL value in the colum category.

In [16]:
# Check NULL values in the column headline
null_values_df=df.filter( (df.headline=='') )
null_values_df.show()
null_values_df.count()

+------------+--------+--------------------+
|    category|headline|   short_description|
+------------+--------+--------------------+
|    POLITICS|        |                    |
|    RELIGION|        |Let everyone not ...|
|   WORLDPOST|        |                    |
|    BUSINESS|        |                    |
|       MEDIA|        |                    |
|QUEER VOICES|        |                    |
+------------+--------+--------------------+



6

There are 6 NULL values in the colum headline

In [17]:
# Check NULL values in the column short_description
null_values_df=df.filter( (df.short_description=='') )
null_values_df.show()
null_values_df.count()

+--------------+--------------------+-----------------+
|      category|            headline|short_description|
+--------------+--------------------+-----------------+
|        COMEDY|   The Big Blue Wave|                 |
|    WORLD NEWS|Inside Rukban Cam...|                 |
|    WORLD NEWS|Syrian Refugees R...|                 |
|         TASTE|Your Guide To The...|                 |
|        COMEDY|    The Bechdel Test|                 |
|         WOMEN|Note: This conten...|                 |
|         WOMEN|Note: This conten...|                 |
|      POLITICS|Fired U.S. Attorn...|                 |
|HEALTHY LIVING|Love Needs To Be ...|                 |
|     WORLDPOST|A New American St...|                 |
|         GREEN|A Price On Carbon...|                 |
|      POLITICS|Geoff Cowan, Form...|                 |
|      POLITICS|Donald Trump Only...|                 |
|     WORLDPOST|Islamic Republic ...|                 |
|      BUSINESS|Price Wrong and L...|           

19712

There are  19712 NULL values in the colum short_description

In short, headline and short_description are the only columns which contains NULL values.

Now let us show the rows with NULL value in both headline and short_description columns


In [18]:
# Find NULL in both headline and short_description columns
null_values_df=df.filter( (df.headline=='') & (df.short_description=='' ))
null_values_df.show()
null_values_df.count()

+------------+--------+-----------------+
|    category|headline|short_description|
+------------+--------+-----------------+
|    POLITICS|        |                 |
|   WORLDPOST|        |                 |
|    BUSINESS|        |                 |
|       MEDIA|        |                 |
|QUEER VOICES|        |                 |
+------------+--------+-----------------+



5

There are 5 matching rows. We will remove them since we cannot perform our task with both headline and description missing.

In [19]:
3 #Select only the rows without NULL values in both headline and short_description columns

df=df.filter( ~((df.headline=='') & (df.short_description=='' )))


In [20]:
# Find rows with NULL value in both headline and short_description columns
df.filter( ((df.headline=='') & (df.short_description=='' ))).count()

0

#### D- Feature engineering

Now that we now that our dataframe doesn't contain any rows with NULL values in both headline and short_description columns, let us add a new colum to the dataframe, called description, with as value the short_description if it is not NULL and the headline if it is NULL

In [21]:
df=df.withColumn("description", when(df.short_description=='', df.short_description).otherwise(df.headline))
df.show(truncate=False)

+--------------+-----------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------+
|category      |headline                                                                                 |short_description                                                                                                                                                                                       |description                                                                              |
+--------------+-----------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------

We can see that the dataset has 6 columns, but we will need only 2: The columns `category` and `short description`. Let us select only these columns

In [22]:
df=df.select(['description','category'])
df.printSchema()
df.show()

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = true)

+--------------------+--------------+
|         description|      category|
+--------------------+--------------+
|Over 4 Million Am...|     U.S. NEWS|
|American Airlines...|     U.S. NEWS|
|23 Of The Funnies...|        COMEDY|
|The Funniest Twee...|     PARENTING|
|Woman Who Called ...|     U.S. NEWS|
|Cleaner Was Dead ...|     U.S. NEWS|
|Reporter Gets Ado...|     U.S. NEWS|
|Puerto Ricans Des...|    WORLD NEWS|
|How A New Documen...|CULTURE & ARTS|
|Biden At UN To Ca...|    WORLD NEWS|
|World Cup Captain...|    WORLD NEWS|
|Man Sets Himself ...|    WORLD NEWS|
|Fiona Threatens T...|    WORLD NEWS|
|Twitch Bans Gambl...|          TECH|
|Virginia Thomas A...|     U.S. NEWS|
|Russian Cosmonaut...|    WORLD NEWS|
|'Reboot' Is A Cle...|CULTURE & ARTS|
|Maury Wills, Base...|        SPORTS|
|4 Russian-Control...|    WORLD NEWS|
|Fiona Barrels Tow...|    WORLD NEWS|
+--------------------+--------------+

In [23]:
df.count()

209522

#### E- Summary

Our preprocessed data now contains 209522 rows, and 2 colums which are `description` and `category`.

Let us copy it into anoter variable.

In [24]:
# Make a copy
df2=df
df2.show()
df2.printSchema()

+--------------------+--------------+
|         description|      category|
+--------------------+--------------+
|Over 4 Million Am...|     U.S. NEWS|
|American Airlines...|     U.S. NEWS|
|23 Of The Funnies...|        COMEDY|
|The Funniest Twee...|     PARENTING|
|Woman Who Called ...|     U.S. NEWS|
|Cleaner Was Dead ...|     U.S. NEWS|
|Reporter Gets Ado...|     U.S. NEWS|
|Puerto Ricans Des...|    WORLD NEWS|
|How A New Documen...|CULTURE & ARTS|
|Biden At UN To Ca...|    WORLD NEWS|
|World Cup Captain...|    WORLD NEWS|
|Man Sets Himself ...|    WORLD NEWS|
|Fiona Threatens T...|    WORLD NEWS|
|Twitch Bans Gambl...|          TECH|
|Virginia Thomas A...|     U.S. NEWS|
|Russian Cosmonaut...|    WORLD NEWS|
|'Reboot' Is A Cle...|CULTURE & ARTS|
|Maury Wills, Base...|        SPORTS|
|4 Russian-Control...|    WORLD NEWS|
|Fiona Barrels Tow...|    WORLD NEWS|
+--------------------+--------------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category:

### Dataset 3 : Environment news dataset


[Link to the dataset (Kaggle)](https://www.kaggle.com/datasets/beridzeg45/guardian-environment-related-news)

#### A- Data exploration

This is a csv file with a header and with values delimited by comass(,).

In [25]:
# Reading the file to get a dataframe
df=spark.read.csv('input/input3.csv',header=True,inferSchema=True)

# Print the number of records
print(df.count(),'rows at all')

# Print the schema of the data
df.printSchema()

# Show the data (first 20 rows)
df.show()

60588 rows at all
root
 |-- Title: string (nullable = true)
 |-- Intro Text: string (nullable = true)
 |-- Authors: string (nullable = true)
 |-- Article Text: string (nullable = true)
 |-- Date Published: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|          Intro Text|             Authors|        Article Text|      Date Published|
+--------------------+--------------------+--------------------+--------------------+--------------------+
| Liz Truss ‘will ...|Tory leadership c...|                NULL|                NULL|                NULL|
|Politics live – l...|                NULL|                NULL|                NULL|                NULL|
|    ","['Rob Davies'|    '@ByRobDavies']"|Liz Truss will si...|          2022-08-30|                NULL|
|Renewed Highland ...|Scottish governme...|['Ewan Murray', '...|It is an area so ...|          2021-03-22|
| Visiting green s.

We remark that the dataset contains **5** fields and **60588** instances.

The fiedls are the following:

- ` Title`: The title of the new.
-` Intro Text`: An introductive text
- `Authors`: The authors of the new
- `Article Text`: The article
- `Date published`: The publication date of the new

#### B- Features selection

Among the 6 columns of our dataset, only 3 are relative to our task. There are `Title`, `Intro Text` and `Article Text`

But since the colum `Article Text` tipically contains too long sentences we won't use it.


Let us select only `Title`, `Intro Text`

In [26]:
# Features selection

df=df.select(['Title','Intro Text'])
#.withColumnsRenamed({'Title':'title','Intro Text':'description'})
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Intro Text: string (nullable = true)



Now the schema of our data is:

- `Title`: The title of the news
-`Intro Text`: A short description of the news

Let us move to the next section, whiich consists to handle missing values.

#### C- Handling missing values

First of all we need to identify the missing values

In [27]:
# Check NULL  values in all columns
null_values_df= df.filter((df['Title'].isNull()) | (df['Intro Text'].isNull()))

null_values_df.show()
null_values_df.count()

+--------------------+--------------------+
|               Title|          Intro Text|
+--------------------+--------------------+
|Politics live – l...|                NULL|
|UK butterfly numb...|                NULL|
|Ukraine | The Bid...|                NULL|
|Politics | Rishi ...|                NULL|
|It’s been over a ...|                NULL|
|     Quick crossword|                NULL|
|   Cryptic crossword|                NULL|
|         ,2022-11-07|                NULL|
|         ,2021-08-31|                NULL|
|Simon Jenkins is ...|                NULL|
|         ,2023-07-31|                NULL|
|What is foot-and-...|                NULL|
|Sign up to receiv...|                NULL|
|Don’t get Down to...|                NULL|
|Human rights expe...|                NULL|
|How criminalisati...|                NULL|
|Follow our Austra...|                NULL|
|Download the free...|                NULL|
|Sign up to receiv...|                NULL|
|                NULL|One of the

15620

In [28]:
null_values_df= df.filter((df['Title'].isNull()) & (df['Intro Text'].isNull()))
null_values_df.show()
null_values_df.count()

+-----+----------+
|Title|Intro Text|
+-----+----------+
| NULL|      NULL|
| NULL|      NULL|
+-----+----------+



2

Basing on these outputs there are 15620 rows containing missing values and two instances having new values for both `Title` and `Text columns`.

Now let us drop these 2 instances.

In [29]:
df=df.na.drop(how='all')

In [30]:
# Check the drop
null_values_df= df.filter((df['Title'].isNull()) & (df['Intro Text'].isNull()))
null_values_df.show()
null_values_df.count()

+-----+----------+
|Title|Intro Text|
+-----+----------+
+-----+----------+



0

#### D- Feature engineering

Now that our dataframe doesn't contain any rows with NULL values in both `Title` and `Intro Text` columns, let us add a new colum to the dataframe, called `description`, with has value the `Intro Text` if it is not NULL and the `Title` if it is NULL

In [31]:
df=df.withColumn("description", when(df['Intro Text'].isNotNull(), df['Intro Text']).otherwise(df['Title']))
df.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Title                                                                                                                                                                               |Intro Text                                                                                                                     

Let us  extact only the relevant column, which is the description.

Also, as we know that all these news are environment news we will add a new colum with as velue, ENVIRONMENT, for all the instances

In [32]:
df=df.select('description').withColumn('category', lit('ENVIRONMENT'))
df.show()
df.printSchema()
df.count()

+--------------------+-----------+
|         description|   category|
+--------------------+-----------+
|Tory leadership c...|ENVIRONMENT|
|Politics live – l...|ENVIRONMENT|
|    '@ByRobDavies']"|ENVIRONMENT|
|Scottish governme...|ENVIRONMENT|
|Positive effects ...|ENVIRONMENT|
|This fantastic ve...|ENVIRONMENT|
|Readers share the...|ENVIRONMENT|
|UK butterfly numb...|ENVIRONMENT|
| pictured on 9 Au...|ENVIRONMENT|
|Tittesworth Reser...|ENVIRONMENT|
|Gardeners at the ...|ENVIRONMENT|
|Danger looms for ...|ENVIRONMENT|
|Some rightwingers...|ENVIRONMENT|
|Researchers now t...|ENVIRONMENT|
| Ben van Beurden ...|ENVIRONMENT|
|Holywell, Flintsh...|ENVIRONMENT|
|In today’s newsle...|ENVIRONMENT|
| the Guardian und...|ENVIRONMENT|
|Ukraine | The Bid...|ENVIRONMENT|
| UK and Canada ha...|ENVIRONMENT|
+--------------------+-----------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = false)



60586

#### E- Summary

Our preprocessed data now contains 60586 rows, and 2 colums which are description and category.

Let us copy it into another variable.

In [33]:
# Make a copy
df3=df
df3.show()
df3.printSchema()

+--------------------+-----------+
|         description|   category|
+--------------------+-----------+
|Tory leadership c...|ENVIRONMENT|
|Politics live – l...|ENVIRONMENT|
|    '@ByRobDavies']"|ENVIRONMENT|
|Scottish governme...|ENVIRONMENT|
|Positive effects ...|ENVIRONMENT|
|This fantastic ve...|ENVIRONMENT|
|Readers share the...|ENVIRONMENT|
|UK butterfly numb...|ENVIRONMENT|
| pictured on 9 Au...|ENVIRONMENT|
|Tittesworth Reser...|ENVIRONMENT|
|Gardeners at the ...|ENVIRONMENT|
|Danger looms for ...|ENVIRONMENT|
|Some rightwingers...|ENVIRONMENT|
|Researchers now t...|ENVIRONMENT|
| Ben van Beurden ...|ENVIRONMENT|
|Holywell, Flintsh...|ENVIRONMENT|
|In today’s newsle...|ENVIRONMENT|
| the Guardian und...|ENVIRONMENT|
|Ukraine | The Bid...|ENVIRONMENT|
| UK and Canada ha...|ENVIRONMENT|
+--------------------+-----------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = false)



### Dataset 4: Busines and Financial news dataset

[Link to the dataset (Kaggle)](https://www.kaggle.com/datasets/thedevastator/cnbc-business-and-financial-news-dataset-450k)

#### A- Data exploration

This is a csv file with a header and with values delimited by comas(,).

In [34]:
# Reading the file to get a dataframe
df=spark.read.csv('input/input4.csv',header=True,inferSchema=True)

# Print the number of records
print(df.count(),'rows at all')

# Print the schema of the data
df.printSchema()

# Show the data (first 20 rows)
df.show()

1234 rows at all
root
 |-- index: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- published_at: string (nullable = true)
 |-- author: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- header_image: string (nullable = true)
 |-- raw_description: string (nullable = true)
 |-- description: string (nullable = true)
 |-- scraped_at: string (nullable = true)

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        index|               title|                 url|        published_at|              author|           publisher|   short_description|            keywords|        header_image|     raw_description|         description|        

We remark that the dataset contains **12** fields and **1234** instances.

The fiedls are the following:

- ` index`: The identifier of the new.
- ` title`: The title of the new.
- ` url`: The link to the new.
- `published_at`: The publication date of the new
- `author`: The authors of the new
-` publisher`: The publisher of the new
-` short_description`: A short description of the new
-` keywords`: The key words in the new
-` header_image`: The image of the headr of the new
- `raw_description`: The craw description of containing HTML tags
- `description`: The description of the new, with HTML tags
- `scraped_at`: The date and time the article was scraped.




#### B- Features selection

Among the 12 columns of our dataset, only 5 are relative to our task. There are `title`, `short_description`,  `keywords`,  `raw_description` and `description`

Let us display the data by focussing more on the content of these columns.

In [35]:
df.select(['title','short_description','keywords','raw_description','description']).show(25,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

By looking at the table, we can select both with betther interest to the following colums: `short_description`, `keyword` and `title`.

Let us select only them.

In [36]:
# Features selection
df=df.select(['title','short_description','keywords','raw_description','description'])
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- raw_description: string (nullable = true)
 |-- description: string (nullable = true)



Now the schema of our datalooks like:

- `short description`: A short description of the news
- `keywords`: The key words in the new.
- `title`: The title of the news
- `raw_description`: The raw description of the new
- `description`: The description of the new




Let us move to the next section, which consists to handle missing values.

#### C- Handling missing values

First of all we need to identify the missing values

In [37]:
# Check NULL  values in all columns
null_values_df= df.filter((df.short_description.isNull() ) | (df.keywords.isNull()) | (df.title.isNull()) | (df.raw_description.isNull()) | (df.description.isNull()) )

null_values_df.show()
null_values_df.count()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|   short_description|            keywords|     raw_description|         description|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|                NULL|                NULL|                NULL|                NULL|                NULL|
|Articles, Politic...|                NULL|                NULL|                NULL|                NULL|
|Trump: 'Mission a...|                NULL|cnbc, Articles, G...|"<div class=""gro...|<div class=""grou...|
|        22. Hexadite|Founders: Eran Ba...|Articles, Technol...|                NULL|                NULL|
|The Materials Of ...|                NULL|cnbc, Articles, C...|"<div class=""gro...|<div class=""grou...|
|                NULL|                NULL|                NULL|                NULL|                NULL|
| the company is d...|               

637

In [38]:
# Check the instances having new values in both our 5 columns of interest
null_values_df= df.filter((df.short_description.isNull() ) & (df.keywords.isNull()) & (df.title.isNull())  & (df.raw_description.isNull()) & (df.description.isNull()) )

null_values_df.show()
null_values_df.count()

+-----+-----------------+--------+---------------+-----------+
|title|short_description|keywords|raw_description|description|
+-----+-----------------+--------+---------------+-----------+
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|       NULL|
| NULL|             NULL|    NULL|           NULL|     

358

Basing on these outputs there are 627 missing values and 358 instances having NULL values for all the 5 columns

Now let us drop these 358 instances.

In [39]:
df=df.na.drop(how='all')

In [40]:
# Check the drop
null_values_df= df.filter((df.short_description.isNull() ) & (df.keywords.isNull()) & (df.title.isNull())  & (df.raw_description.isNull()) & (df.description.isNull()) )
null_values_df.show()
null_values_df.count()

+-----+-----------------+--------+---------------+-----------+
|title|short_description|keywords|raw_description|description|
+-----+-----------------+--------+---------------+-----------+
+-----+-----------------+--------+---------------+-----------+



0

In [41]:
df.count()

876

In [42]:
df.printSchema()

root
 |-- title: string (nullable = true)
 |-- short_description: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- raw_description: string (nullable = true)
 |-- description: string (nullable = true)



#### D- Feature engineering

Now that we now that our dataframe doesn't contain any rows with NULL values in all our columns of interest, let us add a new colum to the dataframe, called description, with as value the `short_description` if not NULL, else the `keywords` if not NULL, else the `title` if not NULL, else the `raw_description` if NOT NULL, otherwise the `description`.

In [43]:
df=df.withColumn(
    "description",
    when(df.short_description.isNotNull(), df.short_description)
    .when(df.keywords.isNotNull(), df.keywords)
    .when(df.title.isNotNull(), df.title)
    .when(df.raw_description.isNotNull(), df.raw_description)
    .otherwise(df.description)
)

df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Let us  extract only the relevant column, which is the description.

Also, as we know that all these news are financial news we will add a new colum called category with as value, BUSINESS & FINANCE, for all these instances

In [44]:
df=df.select('description').withColumn('category', lit('BUSINESS & FINANCE'))
df.show()
df.printSchema()
df.count()

+--------------------+------------------+
|         description|          category|
+--------------------+------------------+
|This is the daily...|BUSINESS & FINANCE|
|"This commentary ...|BUSINESS & FINANCE|
|Articles, Politic...|BUSINESS & FINANCE|
|"In spring, ambit...|BUSINESS & FINANCE|
|The US government...|BUSINESS & FINANCE|
|cnbc, Articles, G...|BUSINESS & FINANCE|
|"Chevron Chief Ex...|BUSINESS & FINANCE|
|Founders: Eran Ba...|BUSINESS & FINANCE|
|LONDON — European...|BUSINESS & FINANCE|
|"Oppenheimer Asse...|BUSINESS & FINANCE|
|James Gorman, Mor...|BUSINESS & FINANCE|
|"We know that one...|BUSINESS & FINANCE|
|The first U.S. ta...|BUSINESS & FINANCE|
|[The stream is sl...|BUSINESS & FINANCE|
|"Renewable energy...|BUSINESS & FINANCE|
|"Italian oil and ...|BUSINESS & FINANCE|
|"Jim Paulsen, chi...|BUSINESS & FINANCE|
|"CDC scientists s...|BUSINESS & FINANCE|
|"Forbes has ""lea...|BUSINESS & FINANCE|
|"Wal-Mart has a r...|BUSINESS & FINANCE|
+--------------------+------------

876

#### E- Summary

Our preprocessed data now contains 876 rows, and 2 colums which are description and category.

Let us copy it into anoter variable.

In [45]:
# Make a copy
df4=df
df4.show()
df4.printSchema()

+--------------------+------------------+
|         description|          category|
+--------------------+------------------+
|This is the daily...|BUSINESS & FINANCE|
|"This commentary ...|BUSINESS & FINANCE|
|Articles, Politic...|BUSINESS & FINANCE|
|"In spring, ambit...|BUSINESS & FINANCE|
|The US government...|BUSINESS & FINANCE|
|cnbc, Articles, G...|BUSINESS & FINANCE|
|"Chevron Chief Ex...|BUSINESS & FINANCE|
|Founders: Eran Ba...|BUSINESS & FINANCE|
|LONDON — European...|BUSINESS & FINANCE|
|"Oppenheimer Asse...|BUSINESS & FINANCE|
|James Gorman, Mor...|BUSINESS & FINANCE|
|"We know that one...|BUSINESS & FINANCE|
|The first U.S. ta...|BUSINESS & FINANCE|
|[The stream is sl...|BUSINESS & FINANCE|
|"Renewable energy...|BUSINESS & FINANCE|
|"Italian oil and ...|BUSINESS & FINANCE|
|"Jim Paulsen, chi...|BUSINESS & FINANCE|
|"CDC scientists s...|BUSINESS & FINANCE|
|"Forbes has ""lea...|BUSINESS & FINANCE|
|"Wal-Mart has a r...|BUSINESS & FINANCE|
+--------------------+------------

### Dataset 5: Indian financial news articles dataset

[Link to the dataset (Kaggle)](https://www.kaggle.com/datasets/hkapoor/indian-financial-news-articles-20032020)

#### A- Data exploration

This is a csv file with a header and with values delimited by comas(,).

In [46]:
# Reading the file to get a dataframe
df=spark.read.csv('input/input5.csv',header=True,inferSchema=True)

# Print the number of records
print(df.count(),'rows at all')

# Print the schema of the data
df.printSchema()

# Show the data (first 20 rows)
df.show()

52513 rows at all
root
 |-- _c0: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)

+---+--------------------+--------------------+--------------------+
|_c0|                Date|               Title|         Description|
+---+--------------------+--------------------+--------------------+
|  0|May 26, 2020, Tue...|ATMs to become vi...|Close to 14.6 per...|
|  1|May 26, 2020, Tue...|IDFC First Bank s...|V Vaidyanathan, m...|
|  2|May 25, 2020, Monday|Huge scam in YES ...|Rana Kapoor's wif...|
|  3|May 24, 2020, Sunday|Bank of Maharasht...|The bank said it ...|
|  4|May 23, 2020, Sat...|DCB Bank's profit...|Net profit for th...|
|  5|May 23, 2020, Sat...|Bank of Baroda to...|Under the scheme,...|
|  6|May 22, 2020, Friday|Retail, MSME loan...|HDFC Bank cuts ba...|
|  7|May 22, 2020, Friday|RBI forecasts eco...|"The six-member M...|
|  8|May 22, 2020, Friday|RBI extends loan ...|The central bank ..

24/06/26 04:54:57 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Date, Title, Description
 Schema: _c0, Date, Title, Description
Expected: _c0 but found: 
CSV file: file:///home/starias/Big-Data-News-Recommender/notebooks/data_preprocessing/input/input5.csv


We remark that the dataset contains **4** fields and **52513** instances.

The fiedls are the following:

- ` _c0`: The identifier of the new.
- `Date`: The publication date of the new
- ` Title`: The title of the new.
-` Description`: The description of the new

#### B- Features selection

Among the 4 columns of our dataset, only 2 are relative to our task. There are `Title`,  and `description`

Let us display the data by focussing more on the content of these columns.

In [47]:
df.select(['title','description']).show(25,truncate=False)

+---------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                      |description                                                                                                                          |
+---------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+
|ATMs to become virtual bank branches, accept deposits with instant credit  |Close to 14.6 per cent (or 35,000) of the 240,000 ATMs in India are new-age recyclers, even though they have been around for only ...|
|IDFC First Bank seniors to forgo 65% of bonus amid Covid-19 crisis         |V Vaidyanathan, managing director and chief executive, will take 30 per cen

By looking at the table, we can select both, with more interest to the column `description`.

Let us extract only them.

In [48]:
# Features selection
df=df.select(['description','title'])
df.printSchema()

root
 |-- description: string (nullable = true)
 |-- title: string (nullable = true)



Now the schema of our data looks like:

- `description`: A description of the news
- `title`: The title of the news


Let us move to the next section, which consists to handle missing values.

#### C- Handling missing values

First of all we need to identify the missing values

In [49]:
# Check all NULL  values
null_values_df= df.filter((df.description.isNull() )  | (df.title.isNull())  )

null_values_df.show()
null_values_df.count()

+-----------+-----+
|description|title|
+-----------+-----+
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
+-----------+-----+
only showing top 20 rows



3221

In [50]:
# Check the instances having new values in both our 2 columns
null_values_df= df.filter((df.description.isNull() )  & (df.title.isNull())  )

null_values_df.show()
null_values_df.count()

+-----------+-----+
|description|title|
+-----------+-----+
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
|       NULL| NULL|
+-----------+-----+
only showing top 20 rows



2504

Basing on these outputs there are 3221 instances missing values and 2504 instances having NULL values for both `description` and `title` columns


In [51]:
df=df.na.drop(how='all')

In [52]:
# Check the drop
null_values_df= df.filter((df.description.isNull() )  & (df.title.isNull()) )
null_values_df.show()
null_values_df.count()

+-----------+-----+
|description|title|
+-----------+-----+
+-----------+-----+



0

In [53]:
df.printSchema()

root
 |-- description: string (nullable = true)
 |-- title: string (nullable = true)



Now the schema of our data looks like:

- `description`: A description of the news
- `title`: The title of the news


Let us move to the next section, which consists to handle missing values.

#### D- Feature engineering

Now that we now that our dataframe doesn't contain any rows with NULL values in all our columns of interest, let us add a new colum to the dataframe, called description, with as value the `description` if not NULL, and the `title` otherwise.

In [54]:
df=df.withColumn(
    "description",
    when(df.description.isNotNull(), df.description)
    .otherwise(df.title)
)

df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|description                                                                                                                          |title                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|Close to 14.6 per cent (or 35,000) of the 240,000 ATMs in India are new-age recyclers, even though they have been around for only ...|ATMs to become virtual bank branches, accept deposits with instant credit  |
|V Vaidyanathan, managing director and chief executive, will take 30 per cent cut in his compensation, including fixed ...            |IDFC First Bank s

Let us  extract only the relevant column, which is the description.

Also, as we know that all these news are financial news we will add a new colum called category with as value, FINANCE, for all these instances

In [55]:
df=df.select('description').withColumn('category', lit('FINANCE'))
df.show()
df.printSchema()
df.count()

+--------------------+--------+
|         description|category|
+--------------------+--------+
|Close to 14.6 per...| FINANCE|
|V Vaidyanathan, m...| FINANCE|
|Rana Kapoor's wif...| FINANCE|
|The bank said it ...| FINANCE|
|Net profit for th...| FINANCE|
|Under the scheme,...| FINANCE|
|HDFC Bank cuts ba...| FINANCE|
|"The six-member M...| FINANCE|
|The central bank ...| FINANCE|
|Two new series wi...| FINANCE|
|Credit costs coul...| FINANCE|
|Enquiries for fre...| FINANCE|
|While slippages f...| FINANCE|
|The meeting was e...| FINANCE|
|In a Q&A, the len...| FINANCE|
|Loans sanctioned ...| FINANCE|
|Senior bank execu...| FINANCE|
|It may be time to...| FINANCE|
|NBFCs cry foul as...| FINANCE|
|The only way to b...| FINANCE|
+--------------------+--------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = false)



50009

#### E- Summary

Our preprocessed data now contains 50009 rows, and 2 colums which are description and category.

Let us copy it into anoter variable.

In [56]:
# Make a copy
df5=df
df5.show()
df5.printSchema()

+--------------------+--------+
|         description|category|
+--------------------+--------+
|Close to 14.6 per...| FINANCE|
|V Vaidyanathan, m...| FINANCE|
|Rana Kapoor's wif...| FINANCE|
|The bank said it ...| FINANCE|
|Net profit for th...| FINANCE|
|Under the scheme,...| FINANCE|
|HDFC Bank cuts ba...| FINANCE|
|"The six-member M...| FINANCE|
|The central bank ...| FINANCE|
|Two new series wi...| FINANCE|
|Credit costs coul...| FINANCE|
|Enquiries for fre...| FINANCE|
|While slippages f...| FINANCE|
|The meeting was e...| FINANCE|
|In a Q&A, the len...| FINANCE|
|Loans sanctioned ...| FINANCE|
|Senior bank execu...| FINANCE|
|It may be time to...| FINANCE|
|NBFCs cry foul as...| FINANCE|
|The only way to b...| FINANCE|
+--------------------+--------+
only showing top 20 rows

root
 |-- description: string (nullable = true)
 |-- category: string (nullable = false)



## IV- Data integration

First all of we need to join the 5 dataframes

In [57]:
# Join the dataframes
df= df1.union(df2).union(df3).union(df4).union(df5)

Let us cache the dataframe

In [58]:
df=df.cache()

In [59]:
# Preview the dataframe
df.show()
df.count()

+--------------------+--------+
|         description|category|
+--------------------+--------+
|A closer look at ...| SCIENCE|
|An irresistible s...| SCIENCE|
|Artificial intell...| SCIENCE|
|Glaciers Could Ha...| SCIENCE|
|Perseid meteor sh...| SCIENCE|
|NASA Releases In-...| SCIENCE|
|SpaceX, NASA Demo...| SCIENCE|
|Orbital space tou...| SCIENCE|
|Greenland's melti...| SCIENCE|
|NASA invites engi...| SCIENCE|
|Astronomers Detec...| SCIENCE|
|NASA Finds Ammoni...| SCIENCE|
|Russia targets re...| SCIENCE|
|Martian Night Sky...| SCIENCE|
|Energy-Efficient ...| SCIENCE|
|Scientists may ha...| SCIENCE|
|Tradeoff between ...| SCIENCE|
|2020's final Mars...| SCIENCE|
|In rare find, fos...| SCIENCE|
|Nasa SpaceX crew ...| SCIENCE|
+--------------------+--------+
only showing top 20 rows



429767

Befrore any operation let us drop identic records

In [60]:
df=df.dropDuplicates()

Let us group all the news by categories and count the number of instances for each category

In [61]:
categories_counts=df.groupBy(df.category).count()
categories_counts.show()
categories_counts.count()

+------------------+-----+
|          category|count|
+------------------+-----+
|            SPORTS|19175|
|             MEDIA| 2403|
|      BLACK VOICES| 4169|
|          POLITICS|32324|
|              ARTS|  864|
|     THE WORLDPOST| 3662|
|BUSINESS & FINANCE|  722|
|      QUEER VOICES| 5596|
|    CULTURE & ARTS| 1044|
|         PARENTING| 8740|
|             GREEN| 2045|
|     ENTERTAINMENT|29491|
|       ENVIRONMENT|49387|
|         U.S. NEWS| 1377|
|              TECH| 2049|
|          BUSINESS|19793|
|     LATINO VOICES| 1023|
|            COMEDY| 4597|
|    STYLE & BEAUTY| 9319|
|             MONEY| 1754|
+------------------+-----+
only showing top 20 rows



48

We can see that there are 48 categories of news in our data

Let's print them all in descendent order of number of instances

In [62]:
# Now group by categories in desc order of number of instances

df.groupBy(df.category).count().orderBy(col('count').desc()).show(48,truncate=False)

+------------------+-----+
|category          |count|
+------------------+-----+
|ENVIRONMENT       |49387|
|FINANCE           |48970|
|POLITICS          |32324|
|ENTERTAINMENT     |29491|
|BUSINESS          |19793|
|SPORTS            |19175|
|WELLNESS          |17895|
|NATION            |14865|
|WORLD             |14337|
|TECHNOLOGY        |14271|
|HEALTH            |12672|
|TRAVEL            |9399 |
|STYLE & BEAUTY    |9319 |
|PARENTING         |8740 |
|FOOD & DRINK      |6327 |
|QUEER VOICES      |5596 |
|HEALTHY LIVING    |5262 |
|SCIENCE           |5057 |
|COMEDY            |4597 |
|HOME & LIVING     |4212 |
|BLACK VOICES      |4169 |
|THE WORLDPOST     |3662 |
|WEDDINGS          |3594 |
|PARENTS           |3511 |
|DIVORCE           |3420 |
|WORLD NEWS        |3293 |
|WOMEN             |3109 |
|IMPACT            |3083 |
|CRIME             |2831 |
|MEDIA             |2403 |
|WEIRD NEWS        |2317 |
|TECH              |2049 |
|GREEN             |2045 |
|TASTE             |1941 |
|

Let us group some categories in order to reduce the number of categories.

The similar categories will be gouped together:

- Group `"ARTS"`, `"CULTURE & ARTS"`  `"ARTS & CULTURE"`, `"SPORTS"`, `"COMEDY"` and `"ENTERTAINMENT"`, to `"ENTERTAINMENT-ARTS-CULTURE"`
- Group `"MONEY"`, `"BUSINESS"` `"BUSINESS & FINANCE"` to `"BUSINESS-ECONOMY-FINANCE"`
- Group `"NATION"`, `"U.S. NEWS"`,  `"WORLD NEWS"`, `"THE WORLDPOST"`,`"WORLDPOST"`  `"WORLD" to "NATION & WORLD"`
- Group `"SCIENCE"`, `"TECH"` , `"TECHNOLOGY"` to `"SCIENCE & TECHNOLOGY"`
- Group `"COLLEGE"`, `"EDUCATION"` to `"EDUCATION"`
- Group `"HEALTHY LIVING"`, `"HEALTH"` to `"HEALTH & WELLNESS"`
- Group `"PARENTS"`, `"PARENTING"` `"WEDDINGS"`, `"DIVORCE"`, to `"FAMILY & RELATIONSHIP"`
- Group `"GREEN"`, `"ENVIRONMENT"` to  `"ENVIRONMENT"`
- Group `"STYLE"` and  `"STYLE & BEAUTY"`, `"HOME & LIVING"`,  `"STYLE & BEAUTY"` to `"LIFESTYLE"`
- Group `"TASTE"`, `"FOOD & DRINK"` `"TRAVEL"`, to `"TRAVEL &FOOD"`
- Group `"BLACK VOICES"`, `"QUEER VOICES"` `"WOMEN"`, `"LATINO VOICES"` to `"SOCIAL ISSUES"`
- Group `"WEIRD NEWS"`,`"GOOD NEWS"`,`"FIFY"`  to `"MISCELLANEOUS"`


In [63]:
# Apply the grouping
df = df.withColumn(
    "category",
    when(col("category").isin("ARTS", "CULTURE & ARTS", "ARTS & CULTURE", "SPORTS", "COMEDY", "ENTERTAINMENT"), "ENTERTAINMENT-ARTS-CULTURE")
    .when(col("category").isin("MONEY", "BUSINESS", "FINANCE","BUSINESS & FINANCE"), "BUSINESS-ECONOMY-FINANCE")
    .when(col("category").isin("NATION", "U.S. NEWS", "WORLD NEWS", "THE WORLDPOST", "WORLDPOST", "WORLD"), "NATION & WORLD")
    .when(col("category").isin("SCIENCE", "TECH", "TECHNOLOGY","MEDIA"), "SCIENCE-TECHNOLOGY-MEDIA")
    .when(col("category").isin("HEALTHY LIVING", "HEALTH","WELLNESS"), "HEALTH & WELLNESS")
    .when(col("category").isin("PARENTS", "PARENTING", "WEDDINGS", "DIVORCE"), "FAMILY & RELATIONSHIP")
    .when(col("category").isin("GREEN", "ENVIRONMENT"), "ENVIRONMENT")
    .when(col("category").isin("STYLE", "STYLE & BEAUTY", "HOME & LIVING"), "LIFESTYLE")
    .when(col("category").isin("TASTE", "FOOD & DRINK", "TRAVEL"), "TRAVEL & FOOD")
    .when(col("category").isin("BLACK VOICES", "QUEER VOICES", "WOMEN", "LATINO VOICES"), "SOCIAL ISSUES")
    .when(col("category").isin("COLLEGE","EDUCATION"), "EDUCATION")
     .when(col("category").isin( "WEIRD NEWS", "GOOD NEWS", "FIFTY"), "MISCELLANEOUS")
    .otherwise(col("category"))
)

df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+------------------------+
|description                                                                                                                        |category                |
+-----------------------------------------------------------------------------------------------------------------------------------+------------------------+
|Getting Canon with filmmaking                                                                                                      |SCIENCE-TECHNOLOGY-MEDIA|
|Major Windows 10 fail is playing havoc with passwords                                                                              |SCIENCE-TECHNOLOGY-MEDIA|
|Teaming and stream sniping proves to be a problem in FNCS Solos                                                                    |SCIENCE-TECHNOLOGY-MEDIA|
|Samsung Galaxy S20 FE 5G renders reveal compl

In [64]:
categories_counts=df.groupBy(df.category).count()
categories_counts.show(truncate=False)
categories_counts.count()

+--------------------------+-----+
|category                  |count|
+--------------------------+-----+
|FAMILY & RELATIONSHIP     |19265|
|POLITICS                  |32324|
|SOCIAL ISSUES             |13897|
|MISCELLANEOUS             |4400 |
|ENVIRONMENT               |51432|
|SCIENCE-TECHNOLOGY-MEDIA  |23780|
|ENTERTAINMENT-ARTS-CULTURE|56509|
|IMPACT                    |3083 |
|RELIGION                  |1878 |
|BUSINESS-ECONOMY-FINANCE  |71239|
|EDUCATION                 |1825 |
|CRIME                     |2831 |
|LIFESTYLE                 |15096|
|TRAVEL & FOOD             |17667|
|NATION & WORLD            |38777|
|HEALTH & WELLNESS         |35829|
+--------------------------+-----+



16

In [65]:
# Group once again the news in DESC order of counts
categories_counts=df.groupBy(df.category).count()
categories_counts.show(truncate=False)
categories_counts.count()

+--------------------------+-----+
|category                  |count|
+--------------------------+-----+
|FAMILY & RELATIONSHIP     |19265|
|POLITICS                  |32324|
|SOCIAL ISSUES             |13897|
|MISCELLANEOUS             |4400 |
|ENVIRONMENT               |51432|
|SCIENCE-TECHNOLOGY-MEDIA  |23780|
|ENTERTAINMENT-ARTS-CULTURE|56509|
|IMPACT                    |3083 |
|RELIGION                  |1878 |
|BUSINESS-ECONOMY-FINANCE  |71239|
|EDUCATION                 |1825 |
|CRIME                     |2831 |
|LIFESTYLE                 |15096|
|TRAVEL & FOOD             |17667|
|NATION & WORLD            |38777|
|HEALTH & WELLNESS         |35829|
+--------------------------+-----+



16

Now we have 16 categories of news in our data

Let's print them all in descendent order of number of count

In [66]:
# Now group by categories in desc order or count

df.groupBy(df.category).count().orderBy(col('count').desc()).show(truncate=False)

+--------------------------+-----+
|category                  |count|
+--------------------------+-----+
|BUSINESS-ECONOMY-FINANCE  |71239|
|ENTERTAINMENT-ARTS-CULTURE|56509|
|ENVIRONMENT               |51432|
|NATION & WORLD            |38777|
|HEALTH & WELLNESS         |35829|
|POLITICS                  |32324|
|SCIENCE-TECHNOLOGY-MEDIA  |23780|
|FAMILY & RELATIONSHIP     |19265|
|TRAVEL & FOOD             |17667|
|LIFESTYLE                 |15096|
|SOCIAL ISSUES             |13897|
|MISCELLANEOUS             |4400 |
|IMPACT                    |3083 |
|CRIME                     |2831 |
|RELIGION                  |1878 |
|EDUCATION                 |1825 |
+--------------------------+-----+



In order to avoid any confusions, let us drop the instances with the following categories `"MISCELLANEOUS"`, `"SOCIAL ISSUES"`, `"NATION & WORLD"`, `"IMPACT"`

In [67]:
df= df.filter(~col("category").isin("MISCELLANEOUS", "SOCIAL ISSUES", "NATION & WORLD","IMPACT"))

In [68]:
# Now group by categories in desc order or count

df.groupBy(df.category).count().orderBy(col('count').desc()).show(truncate=False)

+--------------------------+-----+
|category                  |count|
+--------------------------+-----+
|BUSINESS-ECONOMY-FINANCE  |71239|
|ENTERTAINMENT-ARTS-CULTURE|56509|
|ENVIRONMENT               |51432|
|HEALTH & WELLNESS         |35829|
|POLITICS                  |32324|
|SCIENCE-TECHNOLOGY-MEDIA  |23780|
|FAMILY & RELATIONSHIP     |19265|
|TRAVEL & FOOD             |17667|
|LIFESTYLE                 |15096|
|CRIME                     |2831 |
|RELIGION                  |1878 |
|EDUCATION                 |1825 |
+--------------------------+-----+



In [69]:
number_of_categories=df.groupBy(df.category).count().orderBy(col('count').desc()).count()
number_of_categories

12

Finally we have 12 categories of news. There are

- BUSINESS-ECONOMY-FINANCE
- ENTERTAINMENT-ARTS-CULTURE
- ENVIRONMENT
- HEALTH & WELLNESS
- POLITICS
- SCIENCE-TECHNOLOGY-MEDIA
- FAMILY & RELATIONSHIP
- TRAVEL & FOOD
- LIFESTYLE
- CRIME
- RELIGION
- EDUCATION

Let us find the number of instances of our dataframe with empty description.

In [70]:
df.filter(df.description=='').count()

27

Let us drop them

In [71]:
df=df.filter(df.description!='')

In order to maintain only meaningfull descriptions of news, let us remove all row with descriptions with less than 15 letters or less than 3 words

In [72]:

min_letters = 15

In [73]:
df.filter(length(df.description) < min_letters).count()

3619

In [74]:
df.filter(length(df.description) < min_letters).show(truncate=False)

+--------------+--------------------------+
|description   |category                  |
+--------------+--------------------------+
|Expired       |SCIENCE-TECHNOLOGY-MEDIA  |
|What Is Mars? |SCIENCE-TECHNOLOGY-MEDIA  |
|Geography quiz|HEALTH & WELLNESS         |
|Rising Kashmir|HEALTH & WELLNESS         |
|Tick bite     |HEALTH & WELLNESS         |
|Horoscope     |ENTERTAINMENT-ARTS-CULTURE|
|Recall Release|BUSINESS-ECONOMY-FINANCE  |
|16. NASA Uncut|SCIENCE-TECHNOLOGY-MEDIA  |
|Shock defeat  |ENTERTAINMENT-ARTS-CULTURE|
|India News    |ENTERTAINMENT-ARTS-CULTURE|
|PG no more?   |ENTERTAINMENT-ARTS-CULTURE|
|Luxury Daily  |SCIENCE-TECHNOLOGY-MEDIA  |
|Good morning  |HEALTH & WELLNESS         |
|Katie Mack    |SCIENCE-TECHNOLOGY-MEDIA  |
|China's growth|BUSINESS-ECONOMY-FINANCE  |
|nordstrommens |BUSINESS-ECONOMY-FINANCE  |
|Bloomberg     |BUSINESS-ECONOMY-FINANCE  |
|Remembered    |ENTERTAINMENT-ARTS-CULTURE|
|Acer Spin 5   |SCIENCE-TECHNOLOGY-MEDIA  |
|Smart Start   |BUSINESS-ECONOMY

In [75]:

# Filter the DataFrame to remove descriptions with fewer than the minimum number of letters
df = df.filter(length(df.description) >= min_letters)


In [76]:


# Set the minimum number of words for a description to be considered valid
min_words = 3



In [77]:
# Filter the DataFrame to remove descriptions with fewer than the minimum number of words
df.filter(size(split(df.description, ' ')) < min_words).count()


737

In [78]:
df.filter(size(split(df.description, ' ')) < min_words).show(truncate=False)

+-------------------------------------------+--------------------------+
|description                                |category                  |
+-------------------------------------------+--------------------------+
|Supplementary Materials                    |HEALTH & WELLNESS         |
|Editors' Choice                            |SCIENCE-TECHNOLOGY-MEDIA  |
|AK-StSen-N-nominated                       |HEALTH & WELLNESS         |
|Anti-dongle Portapalooza                   |SCIENCE-TECHNOLOGY-MEDIA  |
|Sargsian's milestone                       |SCIENCE-TECHNOLOGY-MEDIA  |
|Exercise Enhancement                       |HEALTH & WELLNESS         |
|BMW-M3-Wagon-coming                        |SCIENCE-TECHNOLOGY-MEDIA  |
|AK-House-1-nominated                       |HEALTH & WELLNESS         |
|leaked-video-tour-of-the-5g-galaxy-z-fold-2|SCIENCE-TECHNOLOGY-MEDIA  |
|Procuring vaccines                         |HEALTH & WELLNESS         |
|Mosquito Stowaways                         |HEALTH

In [79]:
# Filter the DataFrame to remove descriptions with fewer than the minimum number of words
df=df.filter(size(split(df.description, ' ')) >= min_words)


In [80]:
df.count()

325292

Now let us label encode our categories, and save the different label identifier with their description, into a JSON file.

In [81]:
indexer = StringIndexer(inputCol="category", outputCol="category_label")
indexer_model = indexer.fit(df)
indexed_df = indexer_model.transform(df)
indexed_df.show()

+--------------------+--------------------+--------------+
|         description|            category|category_label|
+--------------------+--------------------+--------------+
|Getting Canon wit...|SCIENCE-TECHNOLOG...|           5.0|
|Major Windows 10 ...|SCIENCE-TECHNOLOG...|           5.0|
|Teaming and strea...|SCIENCE-TECHNOLOG...|           5.0|
|Samsung Galaxy S2...|SCIENCE-TECHNOLOG...|           5.0|
|Covid 19 coronavi...|   HEALTH & WELLNESS|           3.0|
|Man dies of bubon...|   HEALTH & WELLNESS|           3.0|
|Marvel's first Mu...|SCIENCE-TECHNOLOG...|           5.0|
|Should you be wea...|   HEALTH & WELLNESS|           3.0|
|UCSF study: Few S...|SCIENCE-TECHNOLOG...|           5.0|
|4 thoughts on “A ...|SCIENCE-TECHNOLOG...|           5.0|
|Aberdeen Lockdown...|   HEALTH & WELLNESS|           3.0|
|Japan's Nintendo ...|SCIENCE-TECHNOLOG...|           5.0|
|Large study debun...|   HEALTH & WELLNESS|           3.0|
|Sony's PS4 DualSh...|SCIENCE-TECHNOLOG...|           5.

In [82]:
# Retrieve the labels
labels = indexer_model.labels

# Create the dictionary mapping
label_to_index = {label: index for index, label in enumerate(labels)}
index_to_label = {index: label for index, label in enumerate(labels)}

print("Label to Index Mapping: ", label_to_index)
print("Index to Label Mapping: ", index_to_label)


Label to Index Mapping:  {'BUSINESS-ECONOMY-FINANCE': 0, 'ENTERTAINMENT-ARTS-CULTURE': 1, 'ENVIRONMENT': 2, 'HEALTH & WELLNESS': 3, 'POLITICS': 4, 'SCIENCE-TECHNOLOGY-MEDIA': 5, 'FAMILY & RELATIONSHIP': 6, 'TRAVEL & FOOD': 7, 'LIFESTYLE': 8, 'CRIME': 9, 'RELIGION': 10, 'EDUCATION': 11}
Index to Label Mapping:  {0: 'BUSINESS-ECONOMY-FINANCE', 1: 'ENTERTAINMENT-ARTS-CULTURE', 2: 'ENVIRONMENT', 3: 'HEALTH & WELLNESS', 4: 'POLITICS', 5: 'SCIENCE-TECHNOLOGY-MEDIA', 6: 'FAMILY & RELATIONSHIP', 7: 'TRAVEL & FOOD', 8: 'LIFESTYLE', 9: 'CRIME', 10: 'RELIGION', 11: 'EDUCATION'}


In [83]:
# Save the index-to-label mapping to a JSON file
with open('output/news_categories.json', 'w') as json_file:
    dump(index_to_label, json_file, indent=4)

print("Index to label mapping saved to 'output/news_categories.json'")

Index to label mapping saved to 'output/new_categories.json'


In [84]:
df=indexed_df.select(['description','category_label'])

In [85]:
df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|description                                                                                                                        |category_label|
+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|Getting Canon with filmmaking                                                                                                      |5.0           |
|Major Windows 10 fail is playing havoc with passwords                                                                              |5.0           |
|Teaming and stream sniping proves to be a problem in FNCS Solos                                                                    |5.0           |
|Samsung Galaxy S20 FE 5G renders reveal complete design ahead of launch                                  

In [86]:


category_counts=df.groupby('category_label').count().orderBy(desc('count'))
#category_counts.show()
major_class=category_counts.first()
major,max_num=major_class['category_label'], major_class['count']

category_counts=category_counts.withColumn('remaining_nums',max_num-category_counts['count'])
#category_counts.show()
category_counts=category_counts.filter(category_counts.category_label!=major)
#category_counts.show()
# Adding both new columns in one step
category_counts=category_counts.withColumns({
    "sample_size": when(col("remaining_nums") < col("count"), col("remaining_nums"))
                    .otherwise(col('count')),

    "number_of_copies": when(col("remaining_nums") < col("count"), 1)
                    .otherwise(expr("remaining_nums div count")),
    "remainder": when(col("remaining_nums") < col("count"), 0)
                    .otherwise(col("remaining_nums") % col("count")),
})

category_counts.show()

cls = category_counts.select('category_label').collect()
cls=[row['category_label'] for row in cls]
cls

+--------------+-----+--------------+-----------+----------------+---------+
|category_label|count|remaining_nums|sample_size|number_of_copies|remainder|
+--------------+-----+--------------+-----------+----------------+---------+
|           1.0|56335|         14776|      14776|               1|        0|
|           2.0|48782|         22329|      22329|               1|        0|
|           3.0|35287|         35824|      35287|               1|      537|
|           4.0|32175|         38936|      32175|               1|     6761|
|           5.0|23735|         47376|      23735|               1|    23641|
|           6.0|18875|         52236|      18875|               2|    14486|
|           7.0|17506|         53605|      17506|               3|     1087|
|           8.0|15007|         56104|      15007|               3|    11083|
|           9.0| 2823|         68288|       2823|              24|      536|
|          10.0| 1845|         69266|       1845|              37|     1001|

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]

In [87]:
df.printSchema()

root
 |-- description: string (nullable = true)
 |-- category_label: double (nullable = false)



We remark that our dataset does not have balanced categories numbers. Let us handle this by doing data augmentation

In [88]:
# Download necessary NLTK data
download('averaged_perceptron_tagger')
download('punkt')
download('wordnet')

# Initialize augmenter for synonym replacement using WordNet
aug = naw.SynonymAug(aug_src='wordnet')

# Function to generate augmented sentences
def generate_wordnet_augmented_sentences(original_sentence, num_instances=5):
    augmented_sentences = [aug.augment(original_sentence) for _ in range(num_instances)]
    return augmented_sentences

def augment_data(df, category_counts):
    augmented_df = df

    for class_num_row in df.select('category_label').distinct().collect():
        class_num = class_num_row['category_label']
        print(f"Augmenting data for category: {class_num}")

        # Filter category_counts for the current class_num
        filtered_counts = category_counts.filter(col('category_label') == class_num).select(['remaining_nums', 'sample_size', 'number_of_copies', 'remainder']).collect()

        # Check if filtered_counts is empty (though it shouldn't be based on provided data)
        if len(filtered_counts) == 0:
            print(f"No data found for category: {class_num}")
            continue

        x = filtered_counts[0]  # Assuming there's only one row per category_label

        r, size, num_cp, remainder = x['remaining_nums'], x['sample_size'], x['number_of_copies'], x['remainder']

        # Filter the DataFrame and select the required column
        result = df.filter(col('category_label') == class_num).filter(length(col('description')) >= 5).select('description').collect()[:size]

        # Convert the result to an array of strings
        description_filtered_array = [row['description'] for row in result]

        augmented_descriptions = []
        for desc in description_filtered_array:
            augmented_descriptions.extend(generate_wordnet_augmented_sentences(desc, num_instances=num_cp))

        if remainder != 0:
            for desc in description_filtered_array[:remainder]:
                augmented_descriptions.append(generate_wordnet_augmented_sentences(desc, num_instances=1)[0])

        # Create a list of tuples combining category_label and descriptions
        rows = [(desc, class_num) for desc in augmented_descriptions]

        # Create a new DataFrame from the list of tuples
        new_df = spark.createDataFrame(rows, schema=df.schema)

        # Union the new DataFrame with the original DataFrame
        augmented_df = augmented_df.union(new_df)

    return augmented_df

# Example usage
# Assuming df is your original DataFrame and category_counts is the DataFrame containing category statistics
augmented_df = augment_data(df, category_counts)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/starias/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/starias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/starias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Augmenting data for category: 8.0


Augmenting data for category: 0.0
No data found for category: 0.0
Augmenting data for category: 7.0


Augmenting data for category: 1.0


Augmenting data for category: 4.0


...

Augmenting data for category: 2.0


Augmenting data for category: 10.0


Augmenting data for category: 6.0


Augmenting data for category: 5.0


Augmenting data for category: 9.0


In [89]:
augmented_df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|description                                                                                                                        |category_label|
+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|Getting Canon with filmmaking                                                                                                      |5.0           |
|Major Windows 10 fail is playing havoc with passwords                                                                              |5.0           |
|Teaming and stream sniping proves to be a problem in FNCS Solos                                                                    |5.0           |
|Samsung Galaxy S20 FE 5G renders reveal complete design ahead of launch                                  

In [90]:
augmented_df.count()

853332

In [91]:
#categories_counts.show()

In [92]:
augmented_df.printSchema()

root
 |-- description: string (nullable = true)
 |-- category_label: double (nullable = false)



In [93]:
augmented_df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|description                                                                                                                        |category_label|
+-----------------------------------------------------------------------------------------------------------------------------------+--------------+
|Getting Canon with filmmaking                                                                                                      |5.0           |
|Major Windows 10 fail is playing havoc with passwords                                                                              |5.0           |
|Teaming and stream sniping proves to be a problem in FNCS Solos                                                                    |5.0           |
|Samsung Galaxy S20 FE 5G renders reveal complete design ahead of launch                                  

In [94]:
# Now group by categories in desc order or count

augmented_df.groupBy(augmented_df.category_label).count().orderBy(col('count').desc()).show(13,truncate=False)

+--------------+-----+
|category_label|count|
+--------------+-----+
|8.0           |71111|
|0.0           |71111|
|7.0           |71111|
|1.0           |71111|
|4.0           |71111|
|11.0          |71111|
|3.0           |71111|
|2.0           |71111|
|10.0          |71111|
|6.0           |71111|
|5.0           |71111|
|9.0           |71111|
+--------------+-----+



Now let us perform the following text cleaning tasks on our description column:

We wiil:
 - Remove accents and special characters such as accents, punctuations, to keep only spaces and alpha numerical characters
 - Remove extra spaces
 - Transform the text to lowercase


In [95]:
# Remove accents, and special characters and aso extra spaces from the 'description' column
df_cleaned = augmented_df.withColumn("description",
                           trim(
                               regexp_replace(
                                   regexp_replace(col("description"), r'[^a-zA-Z0-9\s]', ''),
                                   r'\s+', ' ')
                               )
                           )
# Show the result
df_cleaned.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+--------------+
|description                                                                                                                |category_label|
+---------------------------------------------------------------------------------------------------------------------------+--------------+
|Getting Canon with filmmaking                                                                                              |5.0           |
|Major Windows 10 fail is playing havoc with passwords                                                                      |5.0           |
|Teaming and stream sniping proves to be a problem in FNCS Solos                                                            |5.0           |
|Samsung Galaxy S20 FE 5G renders reveal complete design ahead of launch                                                    |5.0           |
|Covid 19 cor

In [96]:
# Transform the cleaned description to lowercase
df_lowercase=df_cleaned.withColumn('description_lowercase',lower(df_cleaned['description']))
df_lowercase.show(truncate=False)
df_lowercase.filter(df_lowercase.description_lowercase.isNull()).count()

+---------------------------------------------------------------------------------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------+
|description                                                                                                                |category_label|description_lowercase                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------+--------------+---------------------------------------------------------------------------------------------------------------------------+
|Getting Canon with filmmaking                                                                                              |5.0           |getting canon with filmmaking                                    

0

In [97]:
# Select our relevant columns
df=df_lowercase.select(['description_lowercase','category_label'])
df.show()
df.count()

+---------------------+--------------+
|description_lowercase|category_label|
+---------------------+--------------+
| getting canon wit...|           5.0|
| major windows 10 ...|           5.0|
| teaming and strea...|           5.0|
| samsung galaxy s2...|           5.0|
| covid 19 coronavi...|           3.0|
| man dies of bubon...|           3.0|
| marvels first mus...|           5.0|
| should you be wea...|           3.0|
| ucsf study few sf...|           5.0|
| 4 thoughts on a t...|           5.0|
| aberdeen lockdown...|           3.0|
| japans nintendo s...|           5.0|
| large study debun...|           3.0|
| sonys ps4 dualsho...|           5.0|
| study shows high ...|           3.0|
| amds nextgen ryze...|           5.0|
| microsoft and tik...|           5.0|
| google might not ...|           5.0|
| amds new patent i...|           5.0|
| people in derry b...|           3.0|
+---------------------+--------------+
only showing top 20 rows



853332

Let us show the number of rows with empty lowercase description

In [98]:
df.filter(df.description_lowercase=='').count()

0

Let us drop them

In [99]:
df=df.filter(df.description_lowercase!='')


Now it is time to perform the following transformations:

- Tokenization:

  Here we will split the text into several elements or tokens

- Lemmatization: We will convert the words into their root form: for example,  is am to be

- Stop words removal: We will remove words that doesn't apport meaningful values to the description: for ex with, at, and, etc.

In [100]:
# Initialize Tokenizer
tokenizer = Tokenizer(inputCol="description_lowercase", outputCol="tokens")

# Transform the DataFrame
df_tokenized = tokenizer.transform(df)

# Show the result
df_tokenized.select("description_lowercase", "tokens").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|description_lowercase                                                                                                      |tokens                                                                                                                                             |
+---------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|getting canon with filmmaking                                                                                              |[getting, canon, with, filmmaking]                   

In [101]:
download('wordnet')
lemmatizer = WordNetLemmatizer()


# Broadcast the lemmatizer to all workers
lemmatizer_broadcast = spark.sparkContext.broadcast(lemmatizer)

# Define a UDF to lemmatize tokens

def lemmatize_tokens(tokens):
    lemmatizer = lemmatizer_broadcast.value
    return [lemmatizer.lemmatize(token) for token in tokens]

lemmatize_udf = udf(lemmatize_tokens, ArrayType(StringType()))

# Apply the lemmatization UDF
df_lemmatized = df_tokenized.withColumn("lemmas", lemmatize_udf(col("tokens")))


# Show the result
df_lemmatized.select("description_lowercase", "tokens", "lemmas").show(truncate=False)

[nltk_data] Downloading package wordnet to /home/starias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


+---------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+
|description_lowercase                                                                                                      |tokens                                                                                                                                             |lemmas                                                                                                                                          |
+---------------------------------------------------------------------------------------------------------------------------+-------------------------------------

In [102]:
# Initialize stop words remover
remover = StopWordsRemover(inputCol="lemmas", outputCol="description_filtered")

# Transform the DataFrame
# Remove stop words
df_no_stopwords = remover.transform(df_lemmatized)

# Show the result
df_no_stopwords.select("description_lowercase", 'tokens','lemmas',"description_filtered").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+
|description_lowercase                                                                                                      |tokens                                                                                                                                             |lemmas                                                                                                                                          |description_filtered            

After all these changes, there may by empty desriptions. Let us check that.

In [103]:
df_no_stopwords.filter(size(df_no_stopwords.description_filtered)==0).count()

23

Let us remove all news with empty description.

In [104]:
df_no_stopwords=df_no_stopwords.filter(size(df_no_stopwords.description_filtered)!=0)

Now let us apply these change to our dataframe and selct only the relevant colums

In [105]:
df=df_no_stopwords.select('category_label','description_filtered')
df.show(truncate=False)

+--------------+----------------------------------------------------------------------------------------------------------------------------------+
|category_label|description_filtered                                                                                                              |
+--------------+----------------------------------------------------------------------------------------------------------------------------------+
|5.0           |[getting, canon, filmmaking]                                                                                                      |
|5.0           |[major, window, 10, fail, playing, havoc, password]                                                                               |
|5.0           |[teaming, stream, sniping, prof, problem, fncs, solo]                                                                             |
|5.0           |[samsung, galaxy, s20, fe, 5g, render, reveal, complete, design, ahead, launch]                 

Our filter description fields is a field with arrays. Let us transform the values to strings separted by spaces

In [106]:
df=df.withColumn("description_filtered", concat_ws(" ", "description_filtered"))
df.show(truncate=False)

+--------------+-------------------------------------------------------------------------------------------------------------+
|category_label|description_filtered                                                                                         |
+--------------+-------------------------------------------------------------------------------------------------------------+
|5.0           |getting canon filmmaking                                                                                     |
|5.0           |major window 10 fail playing havoc password                                                                  |
|5.0           |teaming stream sniping prof problem fncs solo                                                                |
|5.0           |samsung galaxy s20 fe 5g render reveal complete design ahead launch                                          |
|3.0           |covid 19 coronavirus second christchurch retirement village go lockdown                        

Now let us drop the eventual duplicates rows and also the rows containing empty description.

In [107]:
df=df.dropDuplicates().filter(df.description_filtered!='')
df.show()
n=df.count()
n

+--------------+--------------------+
|category_label|description_filtered|
+--------------+--------------------+
|           5.0|uv light kill cor...|
|           3.0|quickly process i...|
|           5.0|class action come...|
|           3.0|talk child kilken...|
|           5.0|2020 hyundai mode...|
|           5.0|kpop group bts la...|
|           1.0|wrestler james ka...|
|           5.0|grounded continue...|
|           3.0|global manual ori...|
|           5.0|rajnish kumar lik...|
|           3.0|temporary closure...|
|           3.0|waterloo region r...|
|           0.0|grocer recall pre...|
|           5.0|clear app cache a...|
|           1.0|mark gatisss stag...|
|           0.0|fitch affirms sab...|
|           5.0|oneplus confirms ...|
|           3.0|speed success dip...|
|           1.0|jadon sancho hint...|
|           5.0|youtube music add...|
+--------------+--------------------+
only showing top 20 rows



793624

In [108]:
# Compute the count and percentage of each category
categories = df.groupBy('category_label').count() \
    .withColumn('percentage', format_number((col('count') / n * 100), 2)) \
    .orderBy(col('percentage').desc())

In [109]:
categories.show(truncate=False)

+--------------+-----+----------+
|category_label|count|percentage|
+--------------+-----+----------+
|0.0           |70909|8.93      |
|1.0           |70288|8.86      |
|2.0           |70272|8.85      |
|4.0           |69428|8.75      |
|3.0           |68664|8.65      |
|5.0           |68409|8.62      |
|8.0           |68041|8.57      |
|6.0           |67607|8.52      |
|7.0           |66327|8.36      |
|9.0           |64251|8.10      |
|11.0          |55259|6.96      |
|10.0          |54169|6.83      |
+--------------+-----+----------+



In [110]:
# Convert to a list and use indexing
categories_list = categories.select('count').collect()

# Get the first count (minimum count)
first_count = categories_list[0][0]

# Get the last count (maximum count)
last_count = categories_list[-1][0]

print(f"First count (minimum count): {first_count}")
print(f"Last count (maximum count): {last_count}")

First count (minimum count): 70909
Last count (maximum count): 54169


In [111]:
classes_len=categories.count()
classes_len

12

Let us balance the dataset by undersampling.

In [112]:
balanced_df=spark.createDataFrame([],df.schema)
balanced_df=balanced_df.cache()
balanced_df.printSchema()

for class_num in range(classes_len):
    print('class:',class_num)
    sub_df=df.filter(df.category_label==class_num).limit(last_count)
    balanced_df=balanced_df.unionAll(sub_df)

root
 |-- category_label: double (nullable = false)
 |-- description_filtered: string (nullable = false)

class: 0
class: 1
class: 2
class: 3
class: 4
class: 5
class: 6
class: 7
class: 8
class: 9
class: 10
class: 11


In [113]:
balanced_df.count()

650028

### Summary

Our final dataset has now:

- 2 columns category_label and description_filtered which represent the category of the news and their filtered description
- 650028 rows
- 12 categories which are balanced

The mapping of numerical values to our categories has been saved to a JSON file.

To handle imbalanced classes we performed data augmentation to have  balanced dataset After permorming data cleaning tasks, we then undersample the to get balanced dataset.

We can now save the final version of our data.

In [114]:
balanced_df.toPandas().to_csv('output/news.csv',index=False)

In [115]:
balanced_df.toPandas().to_parquet('output/news.parquet',index=False)

## V- Summary

We have preprocessed each dataset and then integrated them to get the final dataset that we saved into csv and parket formats.

We can stop the application

In [118]:
sc.stop()